In [3]:
import json

# 합칠 파일 목록
files = [
    "qwen3_company_train_dataset_frontend2.json",
    "qwen3_company_train_dataset_backend.json",
    "qwen3_company_train_dataset_data_ai2.json",
    "qwen3_company_train_dataset_cto3.json",
    "qwen3_company_train_dataset_frontend_informal2.json",
    "qwen3_company_train_dataset_backend_informal2.json",
    "qwen3_company_train_dataset_data_ai_informal2.json",
    "qwen3_company_train_dataset_cto_informal2.json",
]

output_file = "qwen3_company_train_dataset_combined.json"

combined_data = []

# 파일들을 순회하면서 데이터 로드 & 합치기
for file in files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        # 리스트라고 가정하고 합치기
        combined_data.extend(data)


# 합친 데이터 저장
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(combined_data, f, ensure_ascii=False, indent=4)

print(f"{len(files)}개 파일이 성공적으로 합쳐져 {output_file}에 저장되었습니다.")


8개 파일이 성공적으로 합쳐져 qwen3_company_train_dataset_combined.json에 저장되었습니다.


In [2]:
print(len(combined_data))

2450


In [6]:
text = open(output_file, "r", encoding="utf-8", errors="replace").read()
print("BOM present?", text.startswith("\ufeff"))

BOM present? False


In [1]:
from pathlib import Path
import json, codecs

path = "qwen3_company_train_dataset_combined.json"
b = Path(path).read_bytes()

# 1) 정말 BOM 없는지 바이트로 확인
print("UTF-8 BOM?", b.startswith(codecs.BOM_UTF8))  # True면 BOM 있음

# 2) 꼬리 바이트 확인
print("Tail bytes:", b[-16:])  # 맨 끝 16바이트

# 3) 꼬리가 잘렸는지(마지막 0~3바이트 잘라 재시도)
for trim in range(0, 4):
    try:
        s = b[:-trim] if trim else b
        s = s.decode("utf-8")
        print(f"UTF-8 decode OK with trim={trim}")
        break
    except UnicodeDecodeError as e:
        print(f"trim={trim} -> {e}")

# 4) JSON 파싱도 확인 (배열 JSON 가정)
if 's' in locals():
    txt = s.strip().rstrip(", \n\r\t")
    if not (txt.endswith("]") or txt.endswith("}")):
        txt += "]"  # 배열이라면 임시로 닫아줌 (임시 복구)
    obj = json.loads(txt)  # 여기서 에러 나면 구조 자체가 손상
    print(type(obj), len(obj) if isinstance(obj, list) else "ok")

UTF-8 BOM? False
Tail bytes: b'     ]\r\n    }\r\n]'
UTF-8 decode OK with trim=0
<class 'list'> 2450
